In [1]:
# Import Libraries
import pandas as pd
import pickle

In [7]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install torch

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Globals for automatic nature
MAKE_DATA = True
ROWS = 100

In [5]:
# Load data
from data.data import Data

if MAKE_DATA:
    data_raw = Data().get_all_data()
    df = data_raw['MBTI 500']

    df = df[['type', 'posts']]
    df.rename(columns={'posts': 'text'}, inplace=True)
    df = df.sample(n = ROWS, ignore_index = True)

    print(f"MBTI_500 loaded with shape: {df.shape}")


MBTI_500 loaded with shape: (100, 2)


In [6]:
# Bert Preprocessing
from scripts.BERT_preprocessing import training_preprocessing

df_list = df(training_preprocessing)
df_list[0]

/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'torch'

In [ ]:
# Create or Load Models
from model.DL import *



In [ ]:
# Train Model

In [ ]:
# Predict Model